## Calculation of Sequence Properties

This notebook computes sequence descriptors for the IDRs in the SPOT-based set

Author: Giulio Tesei

Contact: giulio.tesei@bio.ku.dk

In [3]:
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools
from localcider.sequenceParameters import SequenceParameters
import time, os, glob
from ast import literal_eval
from joblib import dump, load

def calc_seq_prop(seq_name,df,r):
    """df: DataFrame to be populated with sequence properties
    r: DataFrame of aa-specific parameters"""
        
    fasta = list(df.loc[seq_name].fasta).copy()
    fasta_kappa = fasta.copy()
    N = len(fasta)
    
    # calculate properties that do not depend on charges
    df.loc[seq_name,'fK'] = sum([fasta.count(a) for a in ['K']])/N
    df.loc[seq_name,'fR'] = sum([fasta.count(a) for a in ['R']])/N
    df.loc[seq_name,'fE'] = sum([fasta.count(a) for a in ['E']])/N
    df.loc[seq_name,'fD'] = sum([fasta.count(a) for a in ['D']])/N
    df.loc[seq_name,'faro'] = sum([fasta.count(a) for a in ['W','Y','F']])/N
    df.loc[seq_name,'mean_lambda'] = np.mean(r.loc[fasta].lambdas)
    pairs = np.array(list(itertools.combinations(fasta,2)))
    pairs_indices = np.array(list(itertools.combinations(range(N),2)))
    # calculate sequence separations
    ij_dist = np.diff(pairs_indices,axis=1).flatten().astype(float)
    # calculate lambda sums
    ll = r.lambdas.loc[pairs[:,0]].values+r.lambdas.loc[pairs[:,1]].values
    # calculate SHD
    beta = -1
    df.loc[seq_name,'shd'] = np.sum(ll*np.power(np.abs(ij_dist),beta))/N
    
    # fix charges
    if df.loc[seq_name,'first'] == 1:
        r.loc['X'] = r.loc[fasta[0]]
        r.loc['X','q'] = r.loc[fasta[0],'q'] + 1.
        fasta[0] = 'X'
        if r.loc['X','q'] > 0:
            fasta_kappa[0] = 'K'
        else:
            fasta_kappa[0] = 'A'
    if df.loc[seq_name,'last'] == df.loc[seq_name,'N_FL']:
        r.loc['Z'] = r.loc[fasta[-1]]
        r.loc['Z','q'] = r.loc[fasta[-1],'q'] - 1.
        fasta[-1] = 'Z'
        if r.loc['Z','q'] < 0:
            fasta_kappa[-1] = 'D'
        else:
            fasta_kappa[-1] = 'A'
            
    # calculate properties that depend on charges    
    pairs = np.array(list(itertools.combinations(fasta,2)))
    # calculate charge products
    qq = r.q.loc[pairs[:,0]].values*r.q.loc[pairs[:,1]].values
    # calculate SCD
    df.loc[seq_name,'scd'] = np.sum(qq*np.sqrt(ij_dist))/N
    SeqOb = SequenceParameters(''.join(fasta_kappa))
    kappa = SeqOb.get_kappa()
    df.loc[seq_name,'kappa'] = 0 if kappa==-1 else kappa
    df.loc[seq_name,'fcr'] = r.q.loc[list(fasta)].abs().mean()
    df.loc[seq_name,'ncpr'] = r.q.loc[list(fasta)].mean()

def is_between_folded(x):
    is_not_100_nterm = x['first'] > 100
    is_not_100_cterm = (x['N_FL'] - x['last'])  > 100
    is_iternal = is_not_100_nterm & is_not_100_cterm
    is_idr_nterm = []
    is_idr_cterm = []
    for ndx in np.append(x['first_by_ID'],x['last_by_ID']):
        is_idr_nterm.append(ndx in range(x['first']-100,x['first']))
        is_idr_cterm.append(ndx in range(x['last']+1,x['last']+101))
    return bool((not np.any(is_idr_nterm)) and (not np.any(is_idr_cterm)) and is_iternal)

In [4]:
# aa-specific properties
r = pd.read_csv('md_simulations/data/residues.csv').set_index('one',drop=False)

# sequences
df_idrome = pd.read_csv('idr_selection/idr_spot.csv.gz',header=0,sep=';')
df_idrome.sort_values('uniprot',inplace=True)
df_idrome.rename(columns={'uniprot':'UniProt_ID',
                 'sequence':'fasta',
                 'nres_unip':'N_FL',
                 'nres_seg':'N',
                 'name':'seq_name'},inplace=True)
df_idrome.set_index('seq_name',inplace=True)

#### IDRs from UniProt IDs unique to SPOT

In [5]:
df_pLDDT = pd.read_csv('idr_selection/idr.csv.gz',header=0,sep=';')
df_SPOT = pd.read_csv('idr_selection/idr_spot.csv.gz',header=0,sep=';')

UniProt_ID_diff = np.unique(np.setxor1d(df_SPOT.uniprot,df_pLDDT.uniprot))
SPOT_unique = df_SPOT[df_SPOT.uniprot.isin(UniProt_ID_diff)].uniprot
df_SPOT[df_SPOT.uniprot.isin(SPOT_unique)].to_csv('idr_selection/idr_spot_unique.csv.gz')

In [6]:
df_SPOT[df_SPOT.uniprot.isin(SPOT_unique)].uniprot.unique().size

531

#### Load conformational properties for IDRs from UniProt IDs unique to SPOT

In [ ]:
df_idrome_conf = pd.read_csv('md_simulations/data/conf_prop_spot_unique.csv.gz',index_col=0)
df_idrome_conf.nu = df_idrome_conf.nu.apply(lambda x: round(x,3))

In [ ]:
conf_prop = ['nu', 'ete2_Rg2', 'S', 'Delta', 'Rg', 'ete', 'rh', 'SPR']
for seq_name in df_idrome_conf.index:
    df_idrome.loc[seq_name,conf_prop] = df_idrome_conf.loc[seq_name,conf_prop]

#### Calculate sequence descriptors

In [ ]:
seq_prop = ['fK', 'fR', 'fE', 'fD', 'faro', 'mean_lambda', 'shd', 'scd', 'kappa', 'fcr', 'ncpr']
for i,seq_name in enumerate(df_idrome.index):
    calc_seq_prop(seq_name,df_idrome,r)
    print(i,end='-')

In [ ]:
# load SPOT-based IDRome database
df_idrome = pd.read_csv('IDRome_DB_SPOT.csv',index_col=0)

#### Include z-scores

In [ ]:
# load z-scores
if not os.path.isfile(f'zscores/data/SPOT_seq_prop.csv.gz'):
    dfs = []
    for filename in glob.glob(f'../_2023_Tesei_IDRome_v4/zscores/data/SPOT/seq_prop_*.csv.gz'):    
        dfs.append(pd.read_csv(filename,index_col=0))
    for filename in glob.glob(f'../_2023_Tesei_IDRome_v4/zscores/data/SPOT_long/seq_prop_*.csv.gz'):    
        dfs.append(pd.read_csv(filename,index_col=0))

    df_seq_prop = pd.concat(dfs)
    
    df_seq_prop = df_seq_prop[df_seq_prop.seq_name.isin(df_idrome.index)]
    
    df_seq_prop['faro'] = df_seq_prop.seq.apply(lambda x: sum(
        [x.count(a) for a in ['W','Y','F']])/len(x))
    df_seq_prop['fh'] = df_seq_prop.seq.apply(lambda x: sum(
        [x.count(a) for a in ['I','L','M','V']])/len(x))
    
    nonzero_bool = df_seq_prop['faro'].values > .1
    df_seq_prop.loc[~nonzero_bool,'pi_pi'] = 0
    
    nonzero_bool = df_seq_prop['fh'].values > .1
    df_seq_prop.loc[~nonzero_bool,'h_h'] = 0
    
    termini_corr = df_seq_prop.query('termini == True').seq_name
    termini_corr_index = termini_corr.index
    
    # z-scores affected by changes in terminal charges
    termini_seq_props = ['termini', 'seq', 'mu_+', 'mu_-', 'h_+', 'h_-', 
       '+_pi', '+_A', '+_P', '+_G', '-_pi', '-_A', '-_P', '-_G', '+_-', '+_+', '-_-']

    # set correct values to sequences with changes to terminal charges
    df_seq_prop.loc[termini_corr,termini_seq_props] = df_seq_prop.query(
        'termini == True').set_index('seq_name')[termini_seq_props]

    df_seq_prop.drop(termini_corr_index,inplace=True)
    
    df_seq_prop['f+'] = df_seq_prop.seq.apply(lambda x: sum(
        [x.count(a) for a in ['R','K']])/len(x))
    df_seq_prop['f-'] = df_seq_prop.seq.apply(lambda x: sum(
        [x.count(a) for a in ['D','E']])/len(x))
    
    nonzero_bool = (df_seq_prop['f+'].values>.1)&(df_seq_prop['f-'].values>.1)
    df_seq_prop.loc[~nonzero_bool,'+_-'] = 0
    
    nonzero_bool = df_seq_prop['f+'].values > .1
    df_seq_prop.loc[~nonzero_bool,'+_+'] = 0

    nonzero_bool = df_seq_prop['f-'].values > .1
    df_seq_prop.loc[~nonzero_bool,'-_-'] = 0

    df_seq_prop = df_seq_prop.rename({'+_-':'z_delta_+-', 'h_h':'z_omega_h', 
                    'pi_pi':'z_omega_pi', '+_+':'z_omega_+', '-_-':'z_omega_-'},axis=1)

    df_seq_prop.drop(['z_mat', 'z_mat_scr'],axis=1,inplace=True)
    df_seq_prop.to_csv('zscores/data/SPOT_seq_prop.csv.gz')
else:
    df_seq_prop = pd.read_csv('zscores/data/SPOT_seq_prop.csv.gz',index_col=0)
    
df_idrome[['z_delta_+-', 'z_omega_h', 'z_omega_pi', 'z_omega_+', 
           'z_omega_-']] = df_seq_prop[['z_delta_+-', 'z_omega_h', 'z_omega_pi', 
                                        'z_omega_+', 'z_omega_-']]

#### Add $f_\text{domain}$ entries

In [ ]:
df_domains = pd.read_csv('IDRome_DB_SPOT_Domains.csv',index_col=0)
df_idrome['fdomain'] = df_domains['fdomain']

#### Calculate $\nu_{\text{SVR}}$ and $S_\text{conf,SVR}/N$ for all sequences

In [ ]:
model_nu = load('svr_models/svr_model_nu.joblib') 
model_spr = load('svr_models/svr_model_SPR.joblib') 

features_nu = ['scd','shd','kappa','fcr','mean_lambda']
features_spr = ['scd','shd','mean_lambda']

for seq_name in df_idrome.index:
    df_idrome.loc[seq_name,'nu_svr'] = model_nu.predict(df_idrome.loc[seq_name,features_nu].values.reshape(1, -1))
    df_idrome.loc[seq_name,'SPR_svr'] = model_spr.predict(df_idrome.loc[seq_name,features_spr].values.reshape(1, -1))
df_idrome.nu_svr = df_idrome.nu_svr.apply(lambda x: round(x,3))
df_idrome.SPR_svr = df_idrome.SPR_svr.apply(lambda x: round(x,3))

#### Add bool entries for the location of the IDRs within the full-length protein

In [ ]:
first_by_ID = df_idrome.groupby('UniProt_ID')['first'].apply(np.array)
last_by_ID = df_idrome.groupby('UniProt_ID')['last'].apply(np.array)
df_idrome['first_by_ID'] = df_idrome.UniProt_ID.apply(lambda x: first_by_ID.loc[x])
df_idrome['last_by_ID'] = df_idrome.UniProt_ID.apply(lambda x: last_by_ID.loc[x])

df_idrome['is_btw_folded'] = df_idrome.apply(lambda x: is_between_folded(x), axis=1)

df_idrome['is_nterm'] = df_idrome.apply(lambda x: x['first']==1 and x['last']!=x['N_FL'], axis=1)
df_idrome['is_cterm'] = df_idrome.apply(lambda x: x['first']!=1 and x['last']==x['N_FL'], axis=1)

df_idrome['is_idp'] = df_idrome.apply(lambda x: x['first']==1 and x['last']==x['N_FL'], axis=1)

df_idrome.drop(['first_by_ID','last_by_ID'],axis=1,inplace=True)

In [ ]:
df_idrome = df_idrome.rename({'Rg':'Rg/nm','ete':'Ree/nm','rh':'Rh/nm'},axis=1)

In [ ]:
cols = ['UniProt_ID', 'N', 'nu', 'SPR', 'ete2_Rg2', 'S', 'Delta', 'Rg/nm', 'Ree/nm', 'Rh/nm',
       'fK', 'fR', 'fE', 'fD', 'faro', 'mean_lambda', 'shd',
       'scd', 'kappa', 'fcr', 'ncpr', 'fasta', 'is_btw_folded',
       'is_nterm', 'is_cterm', 'is_idp', 'first', 'last', 'N_FL',
       'z_delta_+-', 'z_omega_pi', 'z_omega_+', 'z_omega_-', 'z_omega_h', 'fdomain', 'nu_svr', 'SPR_svr']

In [ ]:
df_idrome[cols].to_csv('IDRome_DB_SPOT.csv')

In [ ]:
df_idrome[cols].to_excel('IDRome_DB_SPOT.xlsx')